# Classifying OUV using NGram features and MLP

## Imports

In [1]:
import sys
sys.executable

'/opt/miniconda3/bin/python'

In [2]:
from argparse import Namespace
from collections import Counter
import json
import os
import re
import string
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix

from scipy.special import softmax

import pickle
import matplotlib.pyplot as plt

import torch.autograd.profiler as profiler

import torchtext
from torchtext.data import get_tokenizer
tokenizer = get_tokenizer('spacy')

In [3]:
print("PyTorch version {}".format(torch.__version__))
print("GPU-enabled installation? {}".format(torch.cuda.is_available()))

PyTorch version 1.7.0
GPU-enabled installation? False


In [4]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cpu


## Data Vectorization Classes

### The Vocabulary

In [5]:
class Vocabulary(object):
    """Class to process text and extract vocabulary for mapping"""

    def __init__(self, token_to_idx=None, add_unk=True, unk_token="<UNK>"):
        """
        Args:
            token_to_idx (dict): a pre-existing map of tokens to indices
            add_unk (bool): a flag that indicates whether to add the UNK token
            unk_token (str): the UNK token to add into the Vocabulary
        """

        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx

        self._idx_to_token = {idx: token 
                              for token, idx in self._token_to_idx.items()}
        
        self._add_unk = add_unk
        self._unk_token = unk_token
        
        self.unk_index = -1
        if add_unk:
            self.unk_index = self.add_token(unk_token) 
        
        
    def to_serializable(self):
        """ returns a dictionary that can be serialized """
        return {'token_to_idx': self._token_to_idx, 
                'add_unk': self._add_unk, 
                'unk_token': self._unk_token}

    @classmethod
    def from_serializable(cls, contents):
        """ instantiates the Vocabulary from a serialized dictionary """
        return cls(**contents)

    def add_token(self, token):
        """Update mapping dicts based on the token.

        Args:
            token (str): the item to add into the Vocabulary
        Returns:
            index (int): the integer corresponding to the token
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
    
    def add_many(self, tokens):
        """Add a list of tokens into the Vocabulary
        
        Args:
            tokens (list): a list of string tokens
        Returns:
            indices (list): a list of indices corresponding to the tokens
        """
        return [self.add_token(token) for token in tokens]

    def lookup_token(self, token):
        """Retrieve the index associated with the token 
          or the UNK index if token isn't present.
        
        Args:
            token (str): the token to look up 
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary) 
              for the UNK functionality 
        """
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

    def lookup_index(self, index):
        """Return the token associated with the index
        
        Args: 
            index (int): the index to look up
        Returns:
            token (str): the token corresponding to the index
        Raises:
            KeyError: if the index is not in the Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]

    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)

    def __len__(self):
        return len(self._token_to_idx)

"""### The Vectorizer"""
def sparse_to_tensor(M):
    """
    input: M is Scipy sparse matrix
    output: pytorch sparse tensor in GPU 
    """
    M = M.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((M.row, M.col))).long()
    values = torch.from_numpy(M.data)
    shape = torch.Size(M.shape)
    Ms = torch.sparse.FloatTensor(indices, values, shape)
    return Ms.to_dense().to(args.device)

### The Vectorizer

In [6]:
def ngrams_iterator(token_list, ngrams):
    """Return an iterator that yields the given tokens and their ngrams.

    Arguments:
        token_list: A list of tokens
        ngrams: the number of ngrams.

    Examples:
        >>> token_list = ['here', 'we', 'are']
        >>> list(ngrams_iterator(token_list, 2))
        >>> ['here', 'here we', 'we', 'we are', 'are']
    """

    def _get_ngrams(n):
        return zip(*[token_list[i:] for i in range(n)])

    for x in token_list:
        yield x
    for n in range(2, ngrams + 1):
        for x in _get_ngrams(n):
            yield ' '.join(x)

In [7]:
# Vectorization parameters
# Range (inclusive) of n-gram sizes for tokenizing text.
NGRAM_RANGE = (1, 2)

# Limit on the number of features. We use the top 20K features.
TOP_K = 20000

# Whether text should be split into word or character n-grams.
# One of 'word', 'char'.
TOKEN_MODE = 'word'

# Minimum document/corpus frequency below which a token will be discarded.
MIN_DOCUMENT_FREQUENCY = 2

def sparse_to_tensor(M):
    """
    input: M is Scipy sparse matrix
    output: pytorch sparse tensor in GPU 
    """
    M = M.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((M.row, M.col))).long()
    values = torch.from_numpy(M.data)
    shape = torch.Size(M.shape)
    Ms = torch.sparse.FloatTensor(indices, values, shape)
    return Ms.to_dense().to(args.device)

class OuvVectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""
    def __init__(self, ouv_vocab, ngrams, vectorizer):
        """
        Args:
            review_vocab (Vocabulary): maps words to integers
        """
        self.ouv_vocab = ouv_vocab
        self.ngrams = ngrams
        self.vectorizer = vectorizer
        
    def vectorize(self, data):
        """Create a tf_idf vector for the ouv data
        
        Args:
            data (str): the ouv description data
            ngrams (int): the maximum ngram value
        Returns:
            tf_idf (np.ndarray): the tf-idf encoding 
        """
        data = [data]
        tf_idf = self.vectorizer.transform(data)
        
        return sparse_to_tensor(tf_idf)[0]

    @classmethod
    def from_dataframe(cls, ouv_df, ngrams, cutoff=5):
        """Instantiate the vectorizer from the dataset dataframe
        
        Args:
            ouv_df (pandas.DataFrame): the ouv dataset
            cutoff (int): the parameter for frequency-based filtering
            ngrams (int): the maximum ngram value
        Returns:
            an instance of the OuvVectorizer
        """
        ouv_vocab = Vocabulary(add_unk=True)
        corpus=[]
        
        # Add top words if count > provided count
        word_counts = Counter()
        for data in ouv_df.data:
            corpus.append(data)
            for word in ngrams_iterator(data.split(' '),ngrams=ngrams):
                if word not in string.punctuation:
                    word_counts[word] += 1
                     
        for word, count in word_counts.items():
            if count > cutoff:
                ouv_vocab.add_token(word)
        
        # Create keyword arguments to pass to the 'tf-idf' vectorizer.
        kwargs = {
                'ngram_range': NGRAM_RANGE,  # Use 1-grams + 2-grams.
                'dtype': 'int32',
                'strip_accents': 'unicode',
                'decode_error': 'replace',
                'analyzer': TOKEN_MODE,  # Split text into word tokens.
                'min_df': MIN_DOCUMENT_FREQUENCY,
        }
        vectorizer = TfidfVectorizer(**kwargs)

        # Learn vocabulary from training texts and vectorize training texts.
        vectorizer.fit_transform(corpus).astype('float32')

        return cls(ouv_vocab, ngrams, vectorizer)

    @classmethod
    def from_serializable(cls, contents, ngrams, vectorizer):
        """Instantiate a OuvVectorizer from a serializable dictionary
        
        Args:
            contents (dict): the serializable dictionary
        Returns:
            an instance of the OuvVectorizer class
        """
        ouv_vocab = Vocabulary.from_serializable(contents['ouv_vocab'])
        
        return cls(ouv_vocab=ouv_vocab, ngrams=ngrams, vectorizer = vectorizer)

    def to_serializable(self):
        """Create the serializable dictionary for caching
        
        Returns:
            contents (dict): the serializable dictionary
        """
        return {'ouv_vocab': self.ouv_vocab.to_serializable()}

### The Dataset

In [8]:
class OuvDataset(Dataset):
    def __init__(self, ouv_df, vectorizer):
        """
        Args:
            ouv_df (pandas.DataFrame): the dataset
            vectorizer (ReviewVectorizer): vectorizer instantiated from dataset
        """
        self.ouv_df = ouv_df
        self._vectorizer = vectorizer

        self.train_df = self.ouv_df[self.ouv_df.split=='train']
        self.train_size = len(self.train_df)

        self.val_df = self.ouv_df[self.ouv_df.split=='dev']
        self.validation_size = len(self.val_df)

        self.test_df = self.ouv_df[self.ouv_df.split=='test']
        self.test_size = len(self.test_df)

        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.validation_size),
                             'test': (self.test_df, self.test_size)}

        self.set_split('train')

    @classmethod
    def load_dataset_and_make_vectorizer(cls, ouv_csv, ngrams, cutoff):
        """Load dataset and make a new vectorizer from scratch
        
        Args:
            ouv_csv (str): location of the dataset
        Returns:
            an instance of OuvDataset
        """
        ouv_df = pd.read_csv(ouv_csv)
        train_ouv_df = ouv_df[ouv_df.split=='train']
        return cls(ouv_df, OuvVectorizer.from_dataframe(train_ouv_df,ngrams=ngrams, cutoff=cutoff))
    
    @classmethod
    def load_dataset_and_load_vectorizer(cls, ouv_csv, vectorizer_filepath, ngrams, vectorizer):
        """Load dataset and the corresponding vectorizer. 
        Used in the case in the vectorizer has been cached for re-use
        
        Args:
            ouv_csv (str): location of the dataset
            vectorizer_filepath (str): location of the saved vectorizer
        Returns:
            an instance of OuvDataset
        """
        ouv_df = pd.read_csv(ouv_csv)
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath, ngrams=ngrams, vectorizer=vectorizer)
        return cls(ouv_df, vectorizer)

    @staticmethod
    def load_vectorizer_only(vectorizer_filepath,ngrams, vectorizer):
        """a static method for loading the vectorizer from file
        
        Args:
            vectorizer_filepath (str): the location of the serialized vectorizer
        Returns:
            an instance of ReviewVectorizer
        """
        with open(vectorizer_filepath) as fp:
            return OuvVectorizer.from_serializable(json.load(fp),ngrams=ngrams, vectorizer=vectorizer)

    def save_vectorizer(self, vectorizer_filepath):
        """saves the vectorizer to disk using json
        
        Args:
            vectorizer_filepath (str): the location to save the vectorizer
        """
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)

    def get_vectorizer(self):
        """ returns the vectorizer """
        return self._vectorizer

    def set_split(self, split="train"):
        """ selects the splits in the dataset using a column in the dataframe 
        
        Args:
            split (str): one of "train", "val", or "test"
        """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        
        Args:
            index (int): the index to the data point 
        Returns:
            a dictionary holding the data point's features (x_data) and component for labels (y_target and y_fuzzy)
        """
        row = self._target_df.iloc[index]

        ouv_vector = \
            self._vectorizer.vectorize(row.data)

        true_label = \
            np.fromstring(row.true[1:-1],dtype=float, sep=' ')
        if len(true_label)==10:
            true_label = np.append(true_label,0.0)
        
        fuzzy_label = \
            np.fromstring(row.fuzzy[1:-1],dtype=float, sep=' ')

        return {'x_data': ouv_vector,
                'y_target': true_label,
                'y_fuzzy': fuzzy_label
               }

    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset
        
        Args:
            batch_size (int)
        Returns:
            number of batches in the dataset
        """
        return len(self) // batch_size  
    
def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"):
    """
    A generator function which wraps the PyTorch DataLoader. It will 
      ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

## The Model: Naive_Bayers_Classifier

In [9]:
class MLPClassifier(nn.Module):
    
    def __init__(self, embedding_size, hidden_dim, num_classes, dropout_p, 
                 pretrained_embeddings=None, padding_idx=0):
        """
        Args:
            embedding_size (int): size of the embedding vectors
            num_embeddings (int): number of embedding vectors
            hidden_dim (int): the size of the hidden dimension
            num_classes (int): the number of classes in classification
            dropout_p (float): a dropout parameter 
            pretrained_embeddings (numpy.array): previously trained word embeddings
                default is None. If provided, 
            padding_idx (int): an index representing a null position
        """
        super(MLPClassifier, self).__init__()

        self._dropout_p = dropout_p
        self.dropout = nn.Dropout(dropout_p)
        self.fc1 = nn.Linear(embedding_size, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x_in, apply_softmax=False):
        """The forward pass of the classifier
        
        Args:
            x_in (torch.Tensor): an input data tensor. 
                x_in.shape should be (batch, dataset._max_seq_length)
            apply_softmax (bool): a flag for the softmax activation
                should be false if used with the Cross Entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch, num_classes)
        """
        
        intermediate_vector = F.relu(self.dropout(self.fc1(x_in)))
        prediction_vector = self.fc2(intermediate_vector)
        
        if apply_softmax:
            prediction_vector = F.softmax(prediction_vector, dim=1)

        return prediction_vector

## Training Routine

### Helper Functions

In [10]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_k_acc_val': 0,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_1_acc': [],
            'train_k_acc': [],
            'train_k_jac': [],
            'val_loss': [],
            'val_1_acc': [],
            'val_k_acc': [],
            'val_k_jac': [],
            'test_loss': -1,
            'test_1_acc': -1,
            'test_k_acc':-1,
            'test_k_jac':-1,
            'model_filename': args.model_state_file}

def update_train_state(args, model, train_state):
    """Handle the training state updates.

    Components:
     - Early Stopping: Prevent overfitting.
     - Model Checkpoint: Model is saved if the model is better

    :param args: main arguments
    :param model: model to train
    :param train_state: a dictionary representing the training state values
    :returns:
        a new train_state
    """

    # Save one model at least
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # Save model if performance improved
    elif train_state['epoch_index'] >= 1:
        acc_tm1, acc_t = train_state['val_k_acc'][-2:]

        # If accuracy worsened
        if acc_t <= train_state['early_stopping_best_k_acc_val']:
            # Update step
            train_state['early_stopping_step'] += 1
        # Loss decreased
        else:
            # Save the best model from sklearn
            if acc_t > train_state['early_stopping_best_k_acc_val']:
                train_state['early_stopping_best_k_acc_val'] = acc_t
                torch.save(model.state_dict(), train_state['model_filename'])
                
            # Reset early stopping step
            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

### Evaluation Metrics

In [11]:
def compute_cross_entropy(y_pred, y_target):
    y_target = y_target.cpu().float()
    y_pred = y_pred.cpu().float()
    criterion = nn.BCEWithLogitsLoss()
    return criterion(y_target, y_pred)

def compute_1_accuracy(y_pred, y_target):
    y_target_indices = y_target.max(dim=1)[1]
    y_pred_indices = y_pred.max(dim=1)[1]
    n_correct = torch.eq(y_pred_indices, y_target_indices).sum().item()
    return n_correct / len(y_pred_indices) * 100

def compute_k_accuracy(y_pred, y_target, k=3):
    y_pred_indices = y_pred.topk(k, dim=1)[1]
    y_target_indices = y_target.max(dim=1)[1]
    n_correct = torch.tensor([y_pred_indices[i] in y_target_indices[i] for i in range(len(y_pred))]).sum().item()
    return n_correct / len(y_pred_indices) * 100

def compute_k_jaccard_index(y_pred, y_target, k=3):
    y_target_indices = y_target.topk(k, dim=1)[1]
    y_pred_indices = y_pred.max(dim=1)[1]
    jaccard = torch.tensor([len(np.intersect1d(y_target_indices[i], y_pred_indices[i]))/
                            len(np.union1d(y_target_indices[i], y_pred_indices[i]))
                            for i in range(len(y_pred))]).sum().item()
    return jaccard / len(y_pred_indices)

def compute_jaccard_index(y_pred, y_target, k=3, multilabel=False):
    
    threshold = 1.0/(k+1)
    threshold_2 = 0.5
    
    if multilabel:
        y_pred_indices = y_pred.gt(threshold_2)
    else:
        y_pred_indices = y_pred.gt(threshold)
    
    y_target_indices = y_target.gt(threshold)
        
    jaccard = ((y_target_indices*y_pred_indices).sum(axis=1)/((y_target_indices+y_pred_indices).sum(axis=1)+1e-8)).sum().item()
    return jaccard / len(y_pred_indices)

In [12]:
def softmax_sensitive(T):
    T = np.exp(T) - np.exp(0) + 1e-9
    if len(T.shape)==1:
        return T/T.sum()
    return  T/(T.sum(axis=1).unsqueeze(1))

In [13]:
def cross_entropy(pred, soft_targets):
    logsoftmax = nn.LogSoftmax(dim=1)
    return torch.mean(torch.sum(- soft_targets * logsoftmax(pred), 1))

In [14]:
# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = args.device
    return torch.from_numpy(df.values).float().to(device)

In [15]:
def get_prior():
    prior = pd.read_csv(args.prior_csv,sep=';',names=classes[:-1], skiprows=1)
    prior['Others'] = 1
    prior = prior.T
    prior['Others'] = 1
    prior = df_to_tensor(prior)
    return prior

In [16]:
def compute_fuzzy_label(y_target, y_fuzzy, fuzzy=False, how='uni', lbd=0):
    '''
    Using two sets of prediction labels and fuzziness parameters to compute the fuzzy label in the form as 
    a distribution over classes
    
    Args:
    y_target (torch.Tensor) of shape (n_batch, n_classes): the true label of the ouv description
    y_fuzzy (torch.Tensor) of shape (n_batch, n_classes): the fuzzy label of the ouv description
    fuzzy (bool): whether or not to turn on the fuzziness option
    how (string): the way fuzziness weights are used, one of the options in {'uni', 'prior'}
    lbd (float): the scaler applied to the fuzziness of the label
    
    Returns:
    A pytorch Tensor of shape (n_batch, n_classes): The processed label in the form of distribution that add to 1
    '''
    assert y_target.shape == y_fuzzy.shape, 'target labels must have the same size'
    assert how in {'uni', 'prior', 'origin'}, '''how must be one of the two options in {'uni', 'prior', 'origin'}'''
    
    if not fuzzy:
        return softmax_sensitive(y_target)
    
    if how == 'uni':
        y_label = y_target + lbd * y_fuzzy
        return softmax_sensitive(y_label)
    
    ### TO DO ###
    elif how == 'prior':
        prior = get_prior()
        y_inter = torch.matmul(y_target.float(),prior)
        y_inter = y_inter/(y_inter.max(dim=1, keepdim=True)[0])
        y_label = y_target + lbd * y_fuzzy * y_inter
        return softmax_sensitive(y_label)
    
    else:
        y_label = y_target + lbd
        return softmax_sensitive(y_label)

In [17]:
def sparse_to_tensor(M):
    """
    input: M is Scipy sparse matrix
    output: pytorch sparse tensor in GPU 
    """
    M = M.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((M.row, M.col))).long()
    values = torch.from_numpy(M.data)
    shape = torch.Size(M.shape)
    Ms = torch.sparse.FloatTensor(indices, values, shape, device=args.device)
    return Ms.to_dense()

### General Utilities

In [18]:
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

### Settings and Some Prep Work

In [19]:
args = Namespace(
    # Data and Path information
    frequency_cutoff=1,
    model_state_file='model.pth',
    ouv_csv='Data/ouv_with_splits_full.csv',
    #ouv_csv='Data/all_with_splits_full.csv',
    prior_csv = 'Data/Coappearance_matrix.csv',
    save_dir='model_storage/ngram/',
    vectorizer_file='vectorizer.json',
    # Model hyper parameters
    ngrams=2,
    hidden_dim=200, 
    # Training hyper parameters
    batch_size=128,
    early_stopping_criteria=5,
    learning_rate=0.0002,
    l2 = 1e-5,
    dropout_p=0.5,
    k = 3,
    fuzzy = True,
    fuzzy_how = 'uni',
    fuzzy_lambda = 0.1,
    num_epochs=100,
    seed=1337,
    # Runtime options
    catch_keyboard_interrupt=True,
    cuda=True,
    expand_filepaths_to_save_dir=True,
    reload_from_files=False,
)

classes = ['Criteria i', 'Criteria ii', 'Criteria iii', 'Criteria iv', 'Criteria v', 'Criteria vi', 
              'Criteria vii', 'Criteria viii', 'Criteria ix', 'Criteria x', 'Others']

if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir,
                                        args.vectorizer_file)

    args.model_state_file = os.path.join(args.save_dir,
                                         args.model_state_file)
    
    print("Expanded filepaths: ")
    print("\t{}".format(args.vectorizer_file))
    print("\t{}".format(args.model_state_file))
    
# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False

print("Using CUDA: {}".format(args.cuda))

args.device = torch.device("cuda" if args.cuda else "cpu")

# Set seed for reproducibility
set_seed_everywhere(args.seed, args.cuda)

# handle dirs
handle_dirs(args.save_dir)

Expanded filepaths: 
	model_storage/ngram/vectorizer.json
	model_storage/ngram/model.pth
Using CUDA: False


## Initialization

In [20]:
set_seed_everywhere(args.seed, args.cuda)
if args.reload_from_files:
    # training from a checkpoint
    dataset = OuvDataset.load_dataset_and_load_vectorizer(args.ouv_csv, args.vectorizer_file)

else:
    # create dataset and vectorizer
    dataset = OuvDataset.load_dataset_and_make_vectorizer(args.ouv_csv, 
                                                          cutoff=args.frequency_cutoff, ngrams=args.ngrams)
    dataset.save_vectorizer(args.vectorizer_file)    

vectorizer = dataset.get_vectorizer()

embedding_size = len(vectorizer.vectorizer.vocabulary_)

classifier = MLPClassifier(embedding_size=embedding_size, 
                            hidden_dim=args.hidden_dim, 
                            num_classes=len(classes), 
                            dropout_p=args.dropout_p)

/opt/miniconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:1799: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


In [21]:
embedding_size

19081

### Training Loop

In [134]:
with profiler.profile(record_shapes=True) as prof:
    with profiler.record_function("model_inference"):
        classifier(X)

In [141]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
        model_inference         6.54%     224.467us        99.33%       3.408ms       3.408ms             1  
            aten::addmm        81.37%       2.792ms        83.10%       2.852ms       1.426ms             2  
          aten::dropout         0.48%      16.322us         7.10%     243.634us     243.634us             1  
       aten::bernoulli_         3.59%     123.054us         3.63%     124.419us     124.419us             1  
                aten::t         1.16%      39.824us         1.70%      58.427us      29.214us             2  
             aten::div_         0.80%      27.575us         1.33%      45.740us      45.740us             1  
          

## Loading Trained Models

### Option 1 LS Model

In [22]:
with open(args.save_dir+'hyperdict_fuzzy.p', 'rb') as fp:
    hyperdict_fuzzy = pickle.load(fp)
    train_state = hyperdict_fuzzy[('uni',0.1)]

In [23]:
classifier.load_state_dict(torch.load(args.save_dir+'1337/model.pth',map_location=torch.device('cpu')))
classifier.eval()

MLPClassifier(
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=19081, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=11, bias=True)
)

### Option 2 Baseline w/o LS

In [402]:
with open(args.save_dir+'hyperdict_fuzzy.p', 'rb') as fp:
    hyperdict_fuzzy = pickle.load(fp)
    train_state = hyperdict_fuzzy[('uni',0)]

In [403]:
classifier.load_state_dict(torch.load(args.save_dir+'baseline/model.pth',map_location=torch.device('cpu')))
classifier.eval()

MLPClassifier(
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=19081, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=11, bias=True)
)

In [24]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(classifier)

3818611

In [404]:
# compute the loss & accuracy on the test set using the best available model
loss_func = cross_entropy
set_seed_everywhere(args.seed, args.cuda)

dataset.set_split('test')
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
running_loss = 0.
running_1_acc = 0.
running_k_acc = 0.
running_k_jac = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    
    # get the data compute fuzzy labels
    X = batch_dict['x_data']

    y_target = batch_dict['y_target']
    y_fuzzy = batch_dict['y_fuzzy']

    Y = compute_fuzzy_label(y_target, y_fuzzy, fuzzy= args.fuzzy, 
                            how=args.fuzzy_how, lbd = args.fuzzy_lambda)

    # compute the output
    with torch.no_grad():
        y_pred = classifier(X)

    # compute the loss
    loss = loss_func(y_pred, Y)
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # compute the accuracy
    acc_1_t = compute_1_accuracy(y_pred, y_target)
    acc_k_t = compute_k_accuracy(y_pred, y_target, args.k)
    jac_k_t = compute_jaccard_index(y_pred, y_target, args.k)

    running_1_acc += (acc_1_t - running_1_acc) / (batch_index + 1)
    running_k_acc += (acc_k_t - running_k_acc) / (batch_index + 1)
    running_k_jac += (jac_k_t - running_k_jac) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_1_acc'] = running_1_acc
train_state['test_k_acc'] = running_k_acc
train_state['test_k_jac'] = running_k_jac

In [392]:
# Result of LS Model
train_state

{'stop_early': True,
 'early_stopping_step': 5,
 'early_stopping_best_k_acc_val': 91.2109375,
 'learning_rate': 0.0002,
 'epoch_index': 32,
 'train_loss': [2.382710953165854,
  2.3501047541321927,
  2.2933758742039005,
  2.2139634354483335,
  2.1189518137611283,
  2.015816979199149,
  1.917336219988604,
  1.8204204257401342,
  1.725383685584271,
  1.6366511834831206,
  1.556673681326131,
  1.472466073606517,
  1.404638742030301,
  1.3314902436662017,
  1.2691519966329905,
  1.206038874454092,
  1.1506658358703707,
  1.1005675252263447,
  1.0470785168565697,
  1.0026070142350851,
  0.960046100542755,
  0.9264518535723673,
  0.8872131928046337,
  0.8561477694927272,
  0.8252839871102317,
  0.7967185736870548,
  0.7708332256268895,
  0.7518785079131212,
  0.7276363425709572,
  0.7088139245532687,
  0.6894438974322717,
  0.6836571894768502,
  0.6769948920907899],
 'train_1_acc': [14.464285714285715,
  17.142857142857142,
  35.982142857142854,
  45.22321428571428,
  44.55357142857144,
  44.

In [405]:
# Result of Baseline
train_state

{'stop_early': True,
 'early_stopping_step': 5,
 'early_stopping_best_k_acc_val': 90.8203125,
 'learning_rate': 0.0002,
 'epoch_index': 32,
 'train_loss': [2.3822073093172866,
  2.3483752532384057,
  2.2898766432460684,
  2.2077932670707567,
  2.1081109083167813,
  1.9974981981136184,
  1.8890756032591272,
  1.7814928880306273,
  1.6739859497765983,
  1.5714231084477355,
  1.4770863319759284,
  1.3769292246986862,
  1.2949315339131129,
  1.2054808567138633,
  1.1291811494103217,
  1.050834673680587,
  0.9816362487981773,
  0.9199762458194751,
  0.8522468257719096,
  0.7965927294180679,
  0.7427864249104039,
  0.6992514694804647,
  0.6490499609507968,
  0.608924275879578,
  0.5695455983635607,
  0.5315245828531986,
  0.49746133108882906,
  0.47098072782707195,
  0.43962891003744736,
  0.413331593465177,
  0.3880896365500506,
  0.37878601054829625,
  0.36913309914749876],
 'train_1_acc': [14.464285714285715,
  17.366071428571427,
  36.54017857142858,
  46.339285714285715,
  45.8035714285

## Inference

In [25]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text

In [26]:
def predict_rating(text, classifier, vectorizer, classes, k=1):
    """Predict the rating of a review
    
    Args:
        text (str): the text of the description
        classifier (ReviewClassifier): the trained model
        vectorizer (ReviewVectorizer): the corresponding vectorizer
        classes (list of str): The name of the ouv classes
        k (int): show the largest k prediction, default to 1
    """
    classifier.eval()
    ouv = preprocess_text(text)
    vectorized_ouv = vectorizer.vectorize(ouv)
    X = vectorized_ouv.view(1,-1)
    with torch.no_grad():
        result = classifier(vectorized_ouv.unsqueeze(0), apply_softmax=True)
    
    if k==1:
        pred_id = result.argmax().item()
        return (classes[pred_id], result[0][pred_id])
    else:
        pred_indices = [i.item() for i in result.topk(k)[1][0]]
        output = []
        for pred_id in pred_indices:
            output.append((classes[pred_id],result[0][pred_id].item()))
        return output

In [27]:
test_ouv = 'this is a very old building dating back to 13th century'

prediction = predict_rating(test_ouv,classifier,vectorizer,classes)
print('{} -> {} with a probability of {:0.2f}'.format(test_ouv, prediction[0],prediction[1]))

this is a very old building dating back to 13th century -> Criteria iv with a probability of 0.31


In [28]:
test_ouv = 'this is a very old building dating back to 13th century'
k=3
predictions = predict_rating(test_ouv,classifier,vectorizer,classes,k)

print("Top {} predictions:".format(k))
print("===================")
for prediction in predictions:
    print('{} -> {} with a probability of {:0.2f}'.format(test_ouv, prediction[0],prediction[1]))

Top 3 predictions:
this is a very old building dating back to 13th century -> Criteria iv with a probability of 0.31
this is a very old building dating back to 13th century -> Criteria iii with a probability of 0.26
this is a very old building dating back to 13th century -> Criteria i with a probability of 0.09


In [29]:
test_ouv = 'The particular layout of the complex is unique to this site'
k=3
predictions = predict_rating(test_ouv,classifier,vectorizer,classes,k)

print("Top {} predictions:".format(k))
print("===================")
for prediction in predictions:
    print('{} -> {} with a probability of {:0.2f}'.format(test_ouv, prediction[0],prediction[1]))

Top 3 predictions:
The particular layout of the complex is unique to this site -> Criteria iv with a probability of 0.27
The particular layout of the complex is unique to this site -> Criteria iii with a probability of 0.24
The particular layout of the complex is unique to this site -> Criteria ii with a probability of 0.16


In [30]:
test_ouv = '''the lagoon of venice also has one of the highest concentrations of masterpieces in the world from 
torcellos cathedral to the church of santa maria della salute . the years of the republics extraordinary golden 
age are represented by monuments of incomparable beauty'''
k=3
predictions = predict_rating(test_ouv,classifier,vectorizer,classes,k)

print("Top {} predictions:".format(k))
print("===================")
for prediction in predictions:
    print('{} -> {} with a probability of {:0.2f}'.format(test_ouv, prediction[0],prediction[1]))

Top 3 predictions:
the lagoon of venice also has one of the highest concentrations of masterpieces in the world from 
torcellos cathedral to the church of santa maria della salute . the years of the republics extraordinary golden 
age are represented by monuments of incomparable beauty -> Criteria i with a probability of 0.32
the lagoon of venice also has one of the highest concentrations of masterpieces in the world from 
torcellos cathedral to the church of santa maria della salute . the years of the republics extraordinary golden 
age are represented by monuments of incomparable beauty -> Criteria iv with a probability of 0.17
the lagoon of venice also has one of the highest concentrations of masterpieces in the world from 
torcellos cathedral to the church of santa maria della salute . the years of the republics extraordinary golden 
age are represented by monuments of incomparable beauty -> Criteria iii with a probability of 0.12


In [31]:
test_ouv = '''the lagoon of venice also has one of the highest concentrations of masterpieces in the world'''
k=3
predictions = predict_rating(test_ouv,classifier,vectorizer,classes,k)

print("Top {} predictions:".format(k))
print("===================")
for prediction in predictions:
    print('{} -> {} with a probability of {:0.2f}'.format(test_ouv, prediction[0],prediction[1]))

Top 3 predictions:
the lagoon of venice also has one of the highest concentrations of masterpieces in the world -> Criteria i with a probability of 0.26
the lagoon of venice also has one of the highest concentrations of masterpieces in the world -> Criteria x with a probability of 0.13
the lagoon of venice also has one of the highest concentrations of masterpieces in the world -> Criteria vii with a probability of 0.13


In [33]:
test_ouv = '''from torcellos cathedral to the church of santa maria della salute'''
k=3
predictions = predict_rating(test_ouv,classifier,vectorizer,classes,k)

print("Top {} predictions:".format(k))
print("===================")
for prediction in predictions:
    print('{} -> {} with a probability of {:0.2f}'.format(test_ouv, prediction[0],prediction[1]))

Top 3 predictions:
from torcellos cathedral to the church of santa maria della salute -> Criteria iv with a probability of 0.28
from torcellos cathedral to the church of santa maria della salute -> Criteria vi with a probability of 0.16
from torcellos cathedral to the church of santa maria della salute -> Criteria ii with a probability of 0.15


In [34]:
test_ouv = '''the years of the republics extraordinary golden age are represented by monuments of incomparable beauty'''
k=3
predictions = predict_rating(test_ouv,classifier,vectorizer,classes,k)

print("Top {} predictions:".format(k))
print("===================")
for prediction in predictions:
    print('{} -> {} with a probability of {:0.2f}'.format(test_ouv, prediction[0],prediction[1]))

Top 3 predictions:
the years of the republics extraordinary golden age are represented by monuments of incomparable beauty -> Criteria iii with a probability of 0.20
the years of the republics extraordinary golden age are represented by monuments of incomparable beauty -> Criteria iv with a probability of 0.20
the years of the republics extraordinary golden age are represented by monuments of incomparable beauty -> Criteria i with a probability of 0.16


In [109]:
import time
class Timer(object):
    def __init__(self, name=None):
        self.name = name

    def __enter__(self):
        self.tstart = time.time()

    def __exit__(self, type, value, traceback):
        if self.name:
            print('[%s]' % self.name,)
        print('Elapsed: %s' % (time.time() - self.tstart))
        
set_seed_everywhere(args.seed, args.cuda)        
test_ouv = 'The particular layout of the complex is unique to this site'
k=3
with Timer():
    predictions = predict_rating(test_ouv,classifier,vectorizer,classes,k=k)

Elapsed: 0.0027370452880859375


## Interpretability

In [164]:
def infer_tokens_importance(vocab, classifier, vectorizer, classes, k=50):
    """Predict the rating of a review
    
    Args:
        vocab (list of str): the whole vocabulary
        classifier (ReviewClassifier): the trained model
        vectorizer (ReviewVectorizer): the corresponding vectorizer
        classes (list of str): The name of the ouv classes
        k (int): show the largest k prediction, default to 1
    """
    classifier.eval()
    X = sparse_to_tensor(vectorizer.vectorizer.transform(list(vocab.keys())))
    
    with torch.no_grad():
        result = classifier(X, apply_softmax=True)
    
    vocab_id = result[1:].topk(k, dim=0)[1]
    vocab_weight = result[1:].topk(k, dim=0)[0]
    return vocab_id, vocab_weight

In [165]:
vocab = vectorizer.vectorizer.vocabulary_
len(vocab)

19081

In [166]:
all_k = infer_tokens_importance(vocab, classifier, vectorizer, classes, k=50)[0]

In [167]:
all_k.shape

torch.Size([50, 11])

In [168]:
id_vocab = {vocab[token]:token for token in vocab.keys()}

In [169]:
def make_top_k_DataFrame(vocab, classifier, vectorizer, classes, k=10):
    
    vocab_id = infer_tokens_importance(vocab, classifier, vectorizer, classes, k)[0]
    df = pd.DataFrame(columns = classes)
    for i in range(len(classes)):
        
        indices = vocab_id[:,i].tolist()
        words = pd.Series([id_vocab[j] for j in indices])
        df[classes[i]] = words
    return df

In [170]:
make_top_k_DataFrame(vocab, classifier, vectorizer, classes, k=20)

,Criteria i,Criteria ii,Criteria iii,Criteria iv,Criteria v,Criteria vi,Criteria vii,Criteria viii,Criteria ix,Criteria x,Others
0,builders,at uxmal,area all,and characteristics,abroad,display of,and patronage,in,arabia and,biblical,jaguars of
1,building and,beki,area an,abraham,be protected,european baroque,of threatened,succession,cross,affected,who
2,bab,at turaif,forested mountains,conditions in,abraham darby,displacement and,meeting place,artistic works,australian tectonic,largest concentration,kindibo
3,buffer against,and society,harmonious interaction,giving rise,coal mining,atoll,is notable,society of,in an,affected by,some are
4,connections with,and st,accentuated,above the,gandharan,displacement,habitat,and historical,arc,considered as,initial
5,baalbek,admirably,accompanied,marae ahu,successive cultures,display,and pelagic,century by,history as,all reflected,known from
6,continuous use,nasrid,is typical,cool temperate,listed,and diffusion,and particularly,improving,the greatest,halls,terraced
7,to sustain,at various,civilization in,active,medieval church,and design,and ideas,mogao,monte san,history that,san
8,buddhist art,application,appreciation of,gjirokastra,gandharan school,and decoration,and many,but are,appropriate,conservation significance,year period
9,nabataean,appears to,shoreline,and chinese,abbey is,below sea,st george,of alpine,outstanding manner,and mostly,the hidden


In [171]:
make_top_k_DataFrame(vocab, classifier, vectorizer, classes, k=50).to_csv(args.save_dir+'top_words.csv')

## Confusion Matrix

In [393]:
dataset.set_split('test')
set_seed_everywhere(args.seed, args.cuda)
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
conf_mat_test = np.zeros((len(classes)-1,len(classes)-1))

In [394]:
for batch_index, batch_dict in enumerate(batch_generator):
    
    # get the data compute fuzzy labels
    X = batch_dict['x_data']

    y_target = batch_dict['y_target']
    y_fuzzy = batch_dict['y_fuzzy']

    Y = compute_fuzzy_label(y_target, y_fuzzy, fuzzy= args.fuzzy, 
                            how=args.fuzzy_how, lbd = args.fuzzy_lambda)

    # compute the output
    y_pred = classifier(X)
    
    conf_mat_test = np.add(conf_mat_test,confusion_matrix(y_target.argmax(axis=1), y_pred.argmax(axis=1),
                                                          labels=range(len(classes)-1)))

In [395]:
conf_mat_test

array([[16.,  6.,  4.,  7.,  0.,  1.,  2.,  0.,  0.,  0.],
       [ 0., 44.,  9., 18.,  2.,  0.,  1.,  0.,  0.,  0.],
       [ 1.,  5., 38., 14.,  4.,  0.,  0.,  0.,  0.,  1.],
       [ 1., 13., 14., 52.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  9., 10.,  7.,  0.,  3.,  1.,  1.,  0.],
       [ 1.,  3., 11.,  9.,  0., 17.,  0.,  0.,  0.,  2.],
       [ 0.,  1.,  0.,  2.,  0.,  0., 34.,  1.,  1.,  3.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  8., 15.,  4.,  2.],
       [ 0.,  0.,  0.,  1.,  0.,  1.,  0.,  3., 28., 14.],
       [ 0.,  0.,  0.,  2.,  0.,  0.,  1.,  0.,  7., 54.]])

In [396]:
dataset.set_split('val')
set_seed_everywhere(args.seed, args.cuda)
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
conf_mat_val = np.zeros((len(classes)-1,len(classes)-1))

In [397]:
for batch_index, batch_dict in enumerate(batch_generator):
    
    # get the data compute fuzzy labels
    X = batch_dict['x_data']

    y_target = batch_dict['y_target']
    y_fuzzy = batch_dict['y_fuzzy']

    Y = compute_fuzzy_label(y_target, y_fuzzy, fuzzy= args.fuzzy, 
                            how=args.fuzzy_how, lbd = args.fuzzy_lambda)

    # compute the output
    y_pred = classifier(X)
    
    conf_mat_val = np.add(conf_mat_val,confusion_matrix(y_target.argmax(axis=1), y_pred.argmax(axis=1),labels=range(len(classes)-1)))

In [398]:
conf_mat_val

array([[10.,  6.,  7.,  8.,  0.,  3.,  1.,  0.,  0.,  0.],
       [ 0., 50.,  5.,  9.,  0.,  1.,  1.,  0.,  0.,  0.],
       [ 3.,  4., 57., 11.,  2.,  2.,  0.,  0.,  0.,  0.],
       [ 1., 11.,  9., 58.,  0.,  2.,  1.,  0.,  0.,  0.],
       [ 0.,  2.,  4.,  8.,  9.,  0.,  1.,  0.,  1.,  0.],
       [ 1.,  7., 10.,  5.,  0., 17.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., 34.,  0.,  3.,  2.],
       [ 0.,  1.,  0.,  3.,  0.,  0.,  7., 22.,  3.,  4.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  9.,  2., 21.,  7.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  2.,  1.,  5., 58.]])

In [399]:
dataset.set_split('train')
set_seed_everywhere(args.seed, args.cuda)
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
conf_mat_train = np.zeros((len(classes)-1,len(classes)-1))

In [400]:
for batch_index, batch_dict in enumerate(batch_generator):
    
    # get the data compute fuzzy labels
    X = batch_dict['x_data']

    y_target = batch_dict['y_target']
    y_fuzzy = batch_dict['y_fuzzy']

    Y = compute_fuzzy_label(y_target, y_fuzzy, fuzzy= args.fuzzy, 
                            how=args.fuzzy_how, lbd = args.fuzzy_lambda)

    # compute the output
    y_pred = classifier(X)
    
    conf_mat_train = np.add(conf_mat_train,confusion_matrix(y_target.argmax(axis=1), y_pred.argmax(axis=1),labels=range(len(classes)-1)))

In [401]:
conf_mat_train

array([[306.,   3.,   5.,  15.,   0.,   0.,   1.,   0.,   0.,   0.],
       [  1., 608.,   5.,  13.,   0.,   1.,   0.,   0.,   0.,   0.],
       [  1.,   2., 634.,   7.,   0.,   1.,   0.,   0.,   0.,   0.],
       [  0.,   4.,   3., 763.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   3.,  12.,  30., 162.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   6.,   4.,   1.,   0., 312.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0., 372.,   0.,   3.,   7.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   4., 254.,   1.,   1.],
       [  0.,   0.,   1.,   1.,   0.,   0.,   1.,   1., 351.,  14.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   3., 563.]])

In [353]:
pd.concat([pd.DataFrame(conf_mat_test),pd.DataFrame(conf_mat_val),pd.DataFrame(conf_mat_train)],axis=1).to_csv(args.save_dir+'confusion_matrix.csv')

In [365]:
pd.concat([pd.DataFrame(conf_mat_test),pd.DataFrame(conf_mat_val),pd.DataFrame(conf_mat_train)],axis=1).to_csv(args.save_dir+'baseline_confusion_matrix.csv')

In [366]:
def per_class_metrics(confusion_matrix, classes):
    '''
    Compute the per class precision, recall, and F1 for all the classes
    
    Args:
    confusion_matrix (np.ndarry) with shape of (n_classes,n_classes): a confusion matrix of interest
    classes (list of str) with shape (n_classes,): The names of classes
    
    Returns:
    metrics_dict (dictionary): a dictionary that records the per class metrics
    '''
    num_class = confusion_matrix.shape[0]
    metrics_dict = {}
    for i in range(num_class):
        key = classes[i]
        temp_dict = {}
        row = confusion_matrix[i,:]
        col = confusion_matrix[:,i]
        val = confusion_matrix[i,i]
        precision = val/row.sum()
        recall = val/col.sum()
        F1 = 2*(precision*recall)/(precision+recall)
        temp_dict['precision'] = precision
        temp_dict['recall'] = recall
        temp_dict['F1'] = F1
        metrics_dict[key] = temp_dict
    
    return metrics_dict

In [367]:
metrics_dict = {}
metrics_dict['test'] = per_class_metrics(conf_mat_test, classes[:-1])
metrics_dict['val'] = per_class_metrics(conf_mat_val, classes[:-1])
metrics_dict['train'] = per_class_metrics(conf_mat_train, classes[:-1])

In [368]:
metrics_df = pd.DataFrame.from_dict({(i,j): metrics_dict[i][j] 
                           for i in metrics_dict.keys() 
                           for j in metrics_dict[i].keys()},
                       orient='index')

In [357]:
metrics_df.to_csv(args.save_dir+'per_class_metrics.csv')

In [369]:
metrics_df.to_csv(args.save_dir+'baseline_per_class_metrics.csv')

## Try on totally Unseen Data

In [406]:
#ouv_csv='Data/ouv_with_splits_full.csv',
new_ouv_csv='Data/sd_full.csv'

In [332]:
def compute_jac_k_accuracy(y_pred, y_target, k=3, multilabel=False):
    
    y_pred_indices = y_pred.topk(k, dim=1)[1]
    y_target_indices = y_target.topk(k, dim=1)[1]
        
    n_correct = torch.tensor([torch.tensor([y_pred_indices[j][i] in y_target_indices[j] for i in range(k)]).sum()>0 
                              for j in range(len(y_pred))]).sum().item()
    return n_correct / len(y_pred_indices) * 100

def compute_jac_1_accuracy(y_pred, y_target, k=3, multilabel=False):
    
    y_pred_indices = y_pred.topk(1, dim=1)[1]
    y_target_indices = y_target.topk(k, dim=1)[1]
        
    n_correct = torch.tensor([torch.tensor([y_pred_indices[j] in y_target_indices[j] for i in range(k)]).sum()>0 
                              for j in range(len(y_pred))]).sum().item()
    return n_correct / len(y_pred_indices) * 100

In [410]:
with Timer():
    loss_func = cross_entropy
    set_seed_everywhere(args.seed, args.cuda)
    train_state = make_train_state(args)
    dataset = OuvDataset.load_dataset_and_load_vectorizer(new_ouv_csv, args.vectorizer_file, 
                                                          ngrams=args.ngrams, vectorizer=vectorizer.vectorizer)

    dataset.set_split('val')
    verbose=False
    try:
      # Iterate over training dataset

        # setup: batch generator, set loss and acc to 0, set train mode on
        dataset.set_split('val')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.0
        running_1_acc = 0.0
        running_k_acc = 0.0
        running_k_jac = 0.0
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):

            # step 2. get the data compute fuzzy labels
            X = batch_dict['x_data']

            y_target = batch_dict['y_target']
            y_fuzzy = batch_dict['y_fuzzy']

            Y = compute_fuzzy_label(y_target, y_fuzzy, fuzzy= args.fuzzy, 
                                    how=args.fuzzy_how, lbd = args.fuzzy_lambda)

            # step 3. compute the output
            with torch.no_grad():
                y_pred = classifier(X)

            # step 4. compute the loss
            loss = loss_func(y_pred, Y)
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # -----------------------------------------
            # compute the accuracies
            acc_1_t = compute_jac_1_accuracy(y_pred, y_target)
            acc_k_t = compute_jac_k_accuracy(y_pred, y_target, args.k)
            jac_k_t = compute_jaccard_index(y_pred, y_target, len(classes))

            running_1_acc += (acc_1_t - running_1_acc) / (batch_index + 1)
            running_k_acc += (acc_k_t - running_k_acc) / (batch_index + 1)
            running_k_jac += (jac_k_t - running_k_jac) / (batch_index + 1)

            # update bar
            if verbose:
                val_bar.set_postfix(loss=running_loss, 
                                acc_1=running_1_acc,
                                acc_k=running_k_acc,
                                jac_k=running_k_jac,
                                epoch=epoch_index)
                val_bar.update()

        train_state['val_loss'].append(running_loss)
        train_state['val_1_acc'].append(running_1_acc)
        train_state['val_k_acc'].append(running_k_acc)
        train_state['val_k_jac'].append(running_k_jac)

    except KeyboardInterrupt:
        print("Exiting loop")
        pass

Elapsed: 6.545941114425659


In [411]:
# LS Model
train_state

{'stop_early': False,
 'early_stopping_step': 0,
 'early_stopping_best_k_acc_val': 0,
 'learning_rate': 0.0002,
 'epoch_index': 0,
 'train_loss': [],
 'train_1_acc': [],
 'train_k_acc': [],
 'train_k_jac': [],
 'val_loss': [1.9158676049580743],
 'val_1_acc': [71.11979166666666],
 'val_k_acc': [95.28645833333333],
 'val_k_jac': [0.3634536236524582],
 'test_loss': -1,
 'test_1_acc': -1,
 'test_k_acc': -1,
 'test_k_jac': -1,
 'model_filename': 'model_storage/ngram/model.pth'}

In [408]:
# Baseline
train_state

{'stop_early': False,
 'early_stopping_step': 0,
 'early_stopping_best_k_acc_val': 0,
 'learning_rate': 0.0002,
 'epoch_index': 0,
 'train_loss': [],
 'train_1_acc': [],
 'train_k_acc': [],
 'train_k_jac': [],
 'val_loss': [2.050477572616485],
 'val_1_acc': [70.49479166666666],
 'val_k_acc': [95.13020833333334],
 'val_k_jac': [0.35870815912882487],
 'test_loss': -1,
 'test_1_acc': -1,
 'test_k_acc': -1,
 'test_k_jac': -1,
 'model_filename': 'model_storage/ngram/model.pth'}

## END